# Prepare data with sequence context

---



## <span class="label label-success"> Analysis </span>

- Write sequence context csv files for naive prediction


---

## <span class="label label-warning">NOTE</span>


---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from Bio import SeqIO
from Bio.Seq import Seq

import os
import subprocess
import time

from IPython.core.display import Image 
from IPython.display import display

import re

In [2]:
workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/prepare_data'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)
os.environ['TMPDIR']='/nagyvinyok/adat84/sotejedlik/ribli/tmp'

In [3]:
#sql exetcuter func
def run_sqlilte3(command,db,output=''):
    start=time.time()
    with open('tempf.sql','w') as tempf:
        tempf.write(command)
        
    if output != '':
        output=' > '+output
    
    try:
        print subprocess.check_output('/usr/bin/sqlite3 '+ db + ' < tempf.sql '+ output,
                                      shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError, e:
        print e.output
    
    subprocess.call(['rm','tempf.sql'])
    print 'It took',int(time.time()-start),'s'

### Select the differentially methylated islands
- Limits: 0.1, 0.3

In [15]:
run_sqlilte3('''
.separator "\t"
.load /home/ribli/tools/sqlite_math_ext/libsqlitefunctions

WITH temp_table AS (
    SELECT probe,mdiff
    FROM meth_median_diff
    WHERE abs(mdiff)>0.3 OR abs(mdiff)<0.1)
    
SELECT n.IlmnID,n.CHR,n.MAPINFO,n.Strand,m.mdiff
FROM ncbi_450_annot AS n
INNER JOIN temp_table AS m ON m.probe=n.IlmnID
WHERE n.Relation_to_UCSC_CpG_Island='Island';

''',db='../db/meth_db',output='diffmet_probes.csv')


It took 6 s


#### Load reference genome for sequence context printing

In [5]:
#load fasta file in one piece
ref_dict=SeqIO.to_dict(SeqIO.parse(
        '/home/ribli/input/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa',"fasta"))

---

## Create csv files to load for naive prediction


Trying to create a large training set, than the not overlapping ones


- test examples are not overlapping with train data
- test examples can overlap with themselves
- train examples can overlap with themselves


Note: hard train,test sets! cannot be changed for cross validation

In [16]:
%%bash
sort -nk2,2 -nk3,3 diffmet_probes.csv > diffmet_probes_sorted.csv

#### Select a random subset for test

In [17]:
#select a random subset for train and validation
N_test=500

#set seed to make the selection reproducible
rng=np.random.RandomState(42)

#load data
#
# there is some weird thing going on with chr sometimes
# integers sometimes characters, this is a workaround
#
all_probes=pd.read_csv('diffmet_probes_sorted.csv',sep='\t',dtype=object)
all_probes.columns=['id','chrom','pos','strand','diff']
all_probes['pos']=np.int32(all_probes['pos'])
all_probes['diff']=np.float32(all_probes['diff'])

#shuffle it
new_idx=rng.permutation(len(all_probes))
all_probes=all_probes.iloc[new_idx,:].reset_index(drop=True)

#select test probes
test_meth_probes=all_probes[all_probes['diff']>=0.3][:N_test]
test_nonmeth_probes=all_probes[all_probes['diff']<=-0.3][:N_test]

#gather all selected probes
test_probes=pd.concat([test_meth_probes,test_nonmeth_probes])

#### Merge filter

In [18]:
# merge filter
window=500

accepted_probes=[]

# sort probes
test_probes=test_probes.sort_values(['chrom','pos']).reset_index(drop=True)
all_probes=all_probes.sort_values(['chrom','pos']).reset_index(drop=True)

#loop over chroms
for chrom in sorted(set(all_probes['chrom'])):
    print chrom,
    #get a smaller df, for the probes on the chrom
    test_probes_tmp=test_probes[test_probes['chrom']==chrom].reset_index(drop=True)
    all_probes_tmp=all_probes[all_probes['chrom']==chrom].reset_index(drop=True)
    
    #get positions of probes
    test_probes_chr=list(test_probes_tmp['pos'])
    all_probes_chr=list(all_probes_tmp['pos'])

    #merge filter the probes
    i,j=0,0
    while(i <len(all_probes_chr) and j+2 < len(test_probes_chr)):
        if (all_probes_chr[i] >= test_probes_chr[j+1]): #adfjust j
            j+=1
        #accept if used position are far enough
        if (test_probes_chr[j] +window < all_probes_chr[i] and
            test_probes_chr[j+1] -window > all_probes_chr[i] ):
            accepted_probes.append(all_probes_tmp.loc[i,:])
        i+=1

1 10 11 12 13 14 15 16 17 18 19 2 20 21 22 3 4 5 6 7 8 9 X Y


In [20]:
accepted_probes=pd.DataFrame(accepted_probes)

#shuffle it
new_idx=rng.permutation(len(accepted_probes))
accepted_probes=accepted_probes.iloc[new_idx,:].reset_index(drop=True)

acc_m_probes=accepted_probes[accepted_probes['diff']>=0.3]
acc_nm_probes=accepted_probes[accepted_probes['diff']<=0.1]

print len(acc_m_probes),len(acc_nm_probes)

3222 103696


#### Select a balanced set

In [23]:
#select equal n probes
N=len(acc_m_probes)
balanced_acc_pr=pd.concat([acc_m_probes[:N],acc_nm_probes[:N]])

#shuffle it again
new_idx=rng.permutation(len(balanced_acc_pr))
balanced_acc_pr=balanced_acc_pr.iloc[new_idx,:].reset_index(drop=True)

#gather and shuffle test too
new_idx=rng.permutation(len(test_probes))
test_probes=test_probes.iloc[new_idx,:].reset_index(drop=True)

### Functions to create vector data

In [27]:
def probe_to_csv(input_df,output_fname,ref_dict,plus=100,minus=100):
    """
    Create relatively small csv files from pandas df of probes.
    
    Used in naive prediction as feauture vectors.
    """

    with open(output_fname,'w') as out_f:
        for i in xrange(len(input_df)):
            try:
                out_f.write(make_naive_line(input_df.iloc[i,:],ref_dict))
            except ValueError,e:
                pass
                
def make_naive_line(line,ref_dict):
    """Transform annotation line into naive feature vector line."""
    probe_id,chrom,pos,strand,beta=line

    #check if the middle is CG 
    cpg1=str(ref_dict[chrom].seq[pos-1])
    cpg2=str(ref_dict[chrom].seq[pos])
    
    if ((cpg1!='c' and cpg1!='C') or (cpg2!='g' and cpg2!='G')) :
        raise ValueError('its no cpg position')

    #seq depending on strand
    if strand=="F":
        out_line = '\t'.join([probe_id]+map(base_to_num,ref_dict[chrom].seq[
                        pos-minus:pos+plus]))
    elif strand=="R":
        out_line = '\t'.join([probe_id]+map(base_to_num,ref_dict[chrom].seq[
                        pos-plus:pos+minus].reverse_complement()))
        
    #label depending on met
    if beta <=0.1:
        out_line+='\t0\n'
    elif beta >=0.3:
        out_line+='\t1\n'
    else:
        raise Exception # bad methylation value
        
    return out_line
                

def base_to_num(base):
    """
    Return a small unique number for each base.
    
    Doing this to turn categorical data into numeric ones.
    Should be checked, if the choice of order makes difference!
    
    """

    if (base =='a' or base=='A'):
        return '1'
    if (base =='c' or base=='C'):
        return '2'
    if (base =='g' or base=='G'):
        return '3'
    if (base =='t' or base=='T'):
        return '4'
    else:
        raise ValueError('strange base in reference genom')

In [28]:
plus,minus=500,500
probe_to_csv(balanced_acc_pr,'diffmet_train_feat_vect.csv',ref_dict,plus=plus,minus=minus)
probe_to_csv(test_probes,'diffmet_test_feat_vect.csv',ref_dict,plus=plus,minus=minus)